# Question and Answering
[Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/03_QuestionAnswering/01_qa_w_rag_claude.ipynb)

In [1]:
from sagemaker import get_execution_role

In [2]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMakerServiceCatalogProductsUseRole


In [3]:
!wget https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
!unzip bedrock-python-sdk.zip -d bedrock-sdk
!rm -rf bedrock-python-sdk.zip

--2023-07-26 15:48:37--  https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
Resolving preview.documentation.bedrock.aws.dev (preview.documentation.bedrock.aws.dev)... 18.65.168.46, 18.65.168.92, 18.65.168.52, ...
Connecting to preview.documentation.bedrock.aws.dev (preview.documentation.bedrock.aws.dev)|18.65.168.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249246443 (238M) [application/zip]
Saving to: ‘bedrock-python-sdk.zip’

100%[======================================>] 249,246,443 26.6MB/s   in 10s    

2023-07-26 15:48:48 (23.9 MB/s) - ‘bedrock-python-sdk.zip’ saved [249246443/249246443]

Archive:  bedrock-python-sdk.zip
   creating: bedrock-sdk/reviews/
  inflating: bedrock-sdk/.unit-crt   
  inflating: bedrock-sdk/awscli-bundle.zip  
  inflating: bedrock-sdk/awscli-1.27.162.tar.gz  
  inflating: bedrock-sdk/botocore-1.29.162.tar.gz  
  inflating: bedrock-sdk/AWSCLISetup.exe  
  inflating: bedrock-sdk/boto3-1.26.

In [11]:
install_needed = True

In [5]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-1.29.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-1.26.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-1.27.162-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 854.4/854.4 kB 5.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for PyYAML~=6.0 from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 82.2 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.173.0-py2.py3-none-any.whl size=1163282 sha256=043232

In [2]:
import os
import sys
module_path = "."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

In [3]:
import boto3
import langchain

In [4]:
bedrock_region = "us-west-2" 
bedrock_config = {
    "region_name":bedrock_region,
    "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
}

In [5]:
boto3_bedrock = bedrock.get_bedrock_client(
    region=bedrock_config["region_name"],
    url_override=bedrock_config["endpoint_url"])
    
modelInfo = boto3_bedrock.list_foundation_models()    
print('models: ', modelInfo)

Create new client
  Using region: us-west-2
assumed_role:  None
boto3 Bedrock client successfully created!
bedrock(https://prod.us-west-2.frontend.bedrock.aws.dev)
models:  {'ResponseMetadata': {'RequestId': '2ac2acef-cdae-451c-b084-39f99943d652', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 26 Jul 2023 15:52:09 GMT', 'content-type': 'application/json', 'content-length': '256', 'connection': 'keep-alive', 'x-amzn-requestid': '2ac2acef-cdae-451c-b084-39f99943d652'}, 'RetryAttempts': 0}, 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large'}, {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium', 'modelId': 'amazon.titan-e1t-medium'}]}


In [6]:
from langchain.llms.bedrock import Bedrock

In [7]:
modelId = 'amazon.titan-tg1-large'
llm = Bedrock(model_id=modelId, client=boto3_bedrock)

In [8]:
llm('Who is the president of usa?')

'\nThe current President of the United States of America is Joe Biden.'

## Kendra

In [18]:
import boto3
from langchain.retrievers import AmazonKendraRetriever

In [19]:
kendraIndex = "50a29d7f-f091-4340-a2cd-fa62f4752e92"

In [20]:
retriever = AmazonKendraRetriever(index_id=kendraIndex)

In [27]:
retriever.get_relevant_documents("what is langchain")

[]

### Customisable option

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
result = qa({"query": query})
print('result: ', result)

In [ ]:
source_documents = result['source_documents']
print(source_documents)

In [ ]:
print('output: ', result['result'])